In [27]:
from PIL import Image
import numpy as np
img_mat = np.zeros((600, 800), dtype = np.uint8)
for i in range(600):
    for j in range(800):
        img_mat[i,j] = (i + j) % 256
img = Image.fromarray(img_mat, mode='L')
img.save("img1.png")

In [103]:
#первоначальный пример
import math
img_mat = np.zeros((200, 200, 3), dtype = np.uint8)
img_mat[:,:] = 255
def dotted_line(img_mat, x0, y0, x1, y1):
        step = 1.0/100
        for t in np.arange(0, 1, step):
            x = round((1-t)*x0 + t * x1)
            y = round((1-t)*y0 + t * y1)
            img_mat[y, x, 0] = 0
for i in range(13):
    x0 = 100
    y0 = 100
    x1 = int(100 + 95 * math.cos((i*2*math.pi)/13))
    y1 = int(100 + 95 * math.sin((i*2*math.pi)/13))
    # dotted_line(img_mat, x0, y0, x1, y1)
    #dotted_line_fixed(img_mat, x0, y0, x1, y1)
    #x_loop_line(img_mat, x0, y0, x1, y1)
    #x_loop_line_fix1(img_mat, x0, y0, x1, y1)
    #x_loop_line_fix2(img_mat, x0, y0, x1, y1)
    #x_loop_line_v2(img_mat, x0, y0, x1, y1)
    #x_loop_line_v2_no_y_calc(img_mat, x0, y0, x1, y1)
    x_loop_final(img_mat, x0, y0, x1, y1)
img = Image.fromarray(img_mat, mode='RGB')
img.save("img10.png")

In [62]:
#изменяем шаг
def dotted_line_fixed(img_mat, x0, y0, x1, y1):
    count = math.sqrt((x0-x1)**2 + (y0-y1)**2)
    step = 1.0/count
    for t in np.arange(0, 1, step):
        x = round((1-t)*x0 + t * x1)
        y = round((1-t)*y0 + t * y1)
        img_mat[y, x, 1] = 0


In [64]:
#цикл по x
def x_loop_line(img_mat, x0, y0, x1, y1):
    for x in range(x0, x1):
        t = (x-x0)/(x1-x0)
        y = round((1-t)*y0 + t * y1)
        img_mat[y,x,1] = 0

In [75]:
#фикс1 цикла х
def x_loop_line_fix1(img_mat, x0, y0, x1, y1):
    if(x1<x0):
        x0, x1 = x1, x0
        y0, y1 = y1, y0
    for x in range(x0, x1):
        t = (x-x0)/(x1-x0)
        y = round((1-t)*y0 + t * y1)
        img_mat[y,x,1] = 0

In [91]:
#фикс2 цикла х
def x_loop_line_fix2(img_mat, x0, y0, x1, y1):
    flag = False
    if(abs(x0-x1) < abs(y0-y1)):
        x0, y0 = y0, x0
        x1, y1 = y1, x1
        flag = True;
    for x in range(x0, x1):
        t = (x-x0)/(x1-x0)
        y = round((1-t)*y0 + t * y1)
        if(flag):
            img_mat[x,y,1] = 0
        else:
            img_mat[y,x,1] = 0

In [94]:
#рабочий вариант
def x_loop_line_v2(img_mat, x0, y0, x1, y1):
    flag = False
    if(abs(x0-x1) < abs(y0-y1)):
        x0, y0 = y0, x0
        x1, y1 = y1, x1
        flag = True;
    if(x1<x0):
        x0, x1 = x1, x0
        y0, y1 = y1, y0
    for x in range(x0, x1):
        t = (x-x0)/(x1-x0)
        y = round((1-t)*y0 + t * y1)
        if(flag):
            img_mat[x,y,1] = 0
        else:
            img_mat[y,x,1] = 0

In [98]:
#убираем из вычисления y
def x_loop_line_v2_no_y_calc(img_mat, x0, y0, x1, y1):
    flag = False
    if(abs(x0-x1) < abs(y0-y1)):
        x0, y0 = y0, x0
        x1, y1 = y1, x1
        flag = True;
    if(x1<x0):
        x0, x1 = x1, x0
        y0, y1 = y1, y0
    y = y0
    dy = abs((y1-y0)/(x1-x0))
    derror = 0.0
    y_update= 1 if y1 > y0 else -1
    for x in range(x0, x1):
        if(flag):
            img_mat[x,y,1] = 0
        else:
            img_mat[y,x,1] = 0
        derror += dy
        if (derror > 0.5):
            derror -= 1.0
            y += y_update

In [102]:
#алгоритм Брезенхема
def x_loop_final(img_mat, x0, y0, x1, y1):
    flag = False
    if(abs(x0-x1) < abs(y0-y1)):
        x0, y0 = y0, x0
        x1, y1 = y1, x1
        flag = True;
    if(x1<x0):
        x0, x1 = x1, x0
        y0, y1 = y1, y0
    y = y0
    dy = 2 * abs((y1-y0))
    derror = 0
    y_update= 1 if y1 > y0 else -1
    for x in range(x0, x1):
        if(flag):
            img_mat[x,y,1] = 0
        else:
            img_mat[y,x,1] = 0
        derror += dy
        if (derror > (x1-x0)):
            derror -= 2 * (x1-x0)
            y += y_update

In [131]:
from PIL import ImageOps
file = open('model_1.obj', 'r')
v = []
f = []
img_mat = np.zeros((1000, 1000), dtype = np.uint8)
for str in file:
    splitted_str = str.split()
    if(splitted_str[0] == 'v'):
        v.append((float(splitted_str[1]),float(splitted_str[2]),float(splitted_str[3])))
for vertex in v:
    img_mat[int(vertex[1] * 5000 + 250), int(vertex[0] * 5000 + 500)] = 255 
img = Image.fromarray(img_mat, mode='L')
im = ImageOps.flip(img)
im.save("img.png")

In [167]:
file = open('model_1.obj', 'r')
v = []
f = []
img_mat = np.zeros((1000, 1000), dtype = np.uint8)
for str in file:
    splitted_str = str.split()
    if(splitted_str[0] == 'v'):
        v.append((float(splitted_str[1]),float(splitted_str[2]),float(splitted_str[3])))
    if(splitted_str[0] == 'f'):
        f.append((int(splitted_str[1].split('/')[0]),int(splitted_str[2].split('/')[0]),int(splitted_str[3].split('/')[0])))
for i in range(len(f)):
        x0 = int(v[f[i][0]-1][0]* 5000 + 500)
        y0 = int(v[f[i][0]-1][1]* 5000 + 250)
        x1 = int(v[f[i][1]-1][0]* 5000 + 500)
        y1 = int(v[f[i][1]-1][1]* 5000 + 250)
        x2 = int(v[f[i][2]-1][0]* 5000 + 500)
        y2 = int(v[f[i][2]-1][1]* 5000 + 250)
        x_loop_finalll(img_mat, x0, y0, x1, y1)
        x_loop_finalll(img_mat, x1, y1, x2, y2)
        x_loop_finalll(img_mat, x2, y2, x0, y0)
img = Image.fromarray(img_mat, mode='L')
im = ImageOps.flip(img)
im.save("img.png")

In [151]:
#алгоритм Брезенхема
def x_loop_finalll(img_mat, x0, y0, x1, y1):
    flag = False
    if(abs(x0-x1) < abs(y0-y1)):
        x0, y0 = y0, x0
        x1, y1 = y1, x1
        flag = True;
    if(x1<x0):
        x0, x1 = x1, x0
        y0, y1 = y1, y0
    y = y0
    dy = 2 * abs((y1-y0))
    derror = 0
    y_update= 1 if y1 > y0 else -1
    for x in range(x0, x1):
        if(flag):
            img_mat[x,y,1] = 255
        else:
            img_mat[y,x,1] = 255
        derror += dy
        if (derror > (x1-x0)):
            derror -= 2 * (x1-x0)
            y += y_update